<a href="https://colab.research.google.com/github/OVP2023/NLP/blob/main/milvus1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pymilvus[milvus_lite]

In [2]:
#Импортируем клиент Milvus плюз база данных milvus_lite

In [3]:
from pymilvus import MilvusClient

client = MilvusClient("/content/drive/MyDrive/RAG/milvus_demo.db")

In [4]:
#!pip freeze

In [5]:
#Из sentence_transformer загружаем модель для формирования эмбеддингов

In [6]:
import torch
from sentence_transformers import SentenceTransformer

In [7]:

# Initialize torch settings for device-agnostic code.
N_GPU = torch.cuda.device_count()
DEVICE = torch.device('cuda:N_GPU' if torch.cuda.is_available() else 'cpu')


# Download the model from huggingface model hub.
model_name = "BAAI/bge-large-en-v1.5"
encoder = SentenceTransformer(model_name, device=DEVICE)


# Get the model parameters and save for later.
EMBEDDING_DIM = encoder.get_sentence_embedding_dimension()
MAX_SEQ_LENGTH_IN_TOKENS = encoder.get_max_seq_length()


# Inspect model parameters.
print(f"model_name: {model_name}")
print(f"EMBEDDING_DIM: {EMBEDDING_DIM}")
print(f"MAX_SEQ_LENGTH: {MAX_SEQ_LENGTH_IN_TOKENS}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_name: BAAI/bge-large-en-v1.5
EMBEDDING_DIM: 1024
MAX_SEQ_LENGTH: 512


In [8]:
#Загружаем файлы текста для формирования эмбеддингов

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import numpy as np

In [10]:
!pip install -U langchain-community

In [11]:
!pip install unstructured

In [12]:
from langchain.document_loaders import DirectoryLoader

In [13]:
# Load HTML files already saved in a local directory
path = "/content/drive/MyDrive/RAG/DOCS/"
global_pattern = '*.html'
loader = DirectoryLoader(path=path, glob=global_pattern)
docs = loader.load()


# Print num documents and a preview.
print(f"loaded {len(docs)} documents")
print(docs[0].page_content)
print(docs[0].metadata)

loaded 5 documents
🚀 Zilliz Cloud: полностью управляемый Milvus - в 10 раз быстрее. Ноль хлопот. Создан для искусственного интеллекта. Попробуйте бесплатно сейчас →
{'source': '/content/drive/MyDrive/RAG/DOCS/Что такое Милвус _ Документация Milvus.html'}


In [14]:
CHUNK_SIZE = 512
chunk_overlap = np.round(CHUNK_SIZE * 0.10, 0)
print(f"chunk_size: {CHUNK_SIZE}, chunk_overlap: {chunk_overlap}")


# Define the splitter.
child_splitter = RecursiveCharacterTextSplitter(
   chunk_size=CHUNK_SIZE,
   chunk_overlap=chunk_overlap)


# Chunk the docs.
chunks = child_splitter.split_documents(docs)
print(f"{len(docs)} docs split into {len(chunks)} child documents.")


# Encoder input is doc.page_content as strings.
list_of_strings = [doc.page_content for doc in chunks if hasattr(doc, 'page_content')]


# Embedding inference using HuggingFace encoder.
embeddings = torch.tensor(encoder.encode(list_of_strings))


# Normalize the embeddings.
embeddings = np.array(embeddings / np.linalg.norm(embeddings))


# Milvus expects a list of `numpy.ndarray` of `numpy.float32` numbers.
converted_values = list(map(np.float32, embeddings))


# Create dict_list for Milvus insertion.
dict_list = []
for chunk, vector in zip(chunks, converted_values):
   # Assemble embedding vector, original text chunk, metadata.
   chunk_dict = {
       'chunk': chunk.page_content,
       'source': chunk.metadata.get('source', ""),
       'vector': vector,
   }
   dict_list.append(chunk_dict)

chunk_size: 512, chunk_overlap: 51.0
5 docs split into 5 child documents.


In [15]:
# Create a collection with flexible schema and AUTOINDEX.
COLLECTION_NAME = "MilvusDocs"
client.create_collection(COLLECTION_NAME,
       EMBEDDING_DIM,
       consistency_level="Eventually",
       auto_id=True,
       overwrite=True)


# Insert data into the Milvus collection.
print("Start inserting entities")
#start_time = time.time()
client.insert(
   COLLECTION_NAME,
   data=dict_list,
   progress_bar=True,
   dim=512)


Start inserting entities


{'insert_count': 5, 'ids': [461061626324844544, 461061626324844545, 461061626324844546, 461061626324844547, 461061626324844548], 'cost': 0}

In [16]:
#Подробное описание коллекции
res = client.describe_collection(
    collection_name="MilvusDocs"
)

print(res)

{'collection_name': 'MilvusDocs', 'auto_id': True, 'num_shards': 0, 'description': '', 'fields': [{'field_id': 100, 'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'params': {}, 'auto_id': True, 'is_primary': True}, {'field_id': 101, 'name': 'vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 1024}}], 'functions': [], 'aliases': [], 'collection_id': 0, 'consistency_level': 0, 'properties': {}, 'num_partitions': 0, 'enable_dynamic_field': True}


In [17]:
SAMPLE_QUESTION = "Hugging Face"


# Embed the question using the same encoder.
query_embeddings = torch.tensor(encoder.encode(SAMPLE_QUESTION))

# Convert the embeddings to list of list of np.float32.
query_embeddings = list(map(np.float32, query_embeddings))

#нормализуем
query_embeddings = np.array(query_embeddings / np.linalg.norm(query_embeddings))

print(len(query_embeddings),query_embeddings)



1024 [ 0.00060368  0.02985209  0.00949713 ... -0.02161123 -0.0020405
 -0.02483699]


In [18]:
res = client.search(
    collection_name="MilvusDocs",
    anns_field="vector",
    data=[query_embeddings],
    limit=14,   #количество выдаваемых записей
    search_params={"metric_type": "COSINE"}      #выбираем косинусное сходство
)

for hits in res:
    for hit in hits:
        print(hit)

{'id': 461061626324844548, 'distance': 0.8314166069030762, 'entity': {}}
{'id': 461061626324844547, 'distance': 0.8314166069030762, 'entity': {}}
{'id': 461061626324844546, 'distance': 0.47015896439552307, 'entity': {}}
{'id': 461061626324844545, 'distance': 0.47015896439552307, 'entity': {}}
{'id': 461061626324844544, 'distance': 0.47015896439552307, 'entity': {}}


In [19]:
#Правильно поставил 2 файла с HF на первые места с 0.83, а три файла с milvus идут за ними с 0.47